# Random forests P22250 model

## Setup

### Imports

In [67]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
import os

## Data

Get PUF records.

In [68]:
train = pd.read_csv('puf80%training')
test = pd.read_csv('puf20%testing')

cols =  ['P22250', 'P22250_sign', 'DSI', 'EIC', 'MARS', 'XTOT', 
         'E00200', 'E00300', 'E00400', 'E00600', 'E00800', 'E00900', 
         'E01100', 'E01400', 'E01500', 'E01700', 'E02100', 'E02300', 
         'E02400', 'E03150', 'E03210', 'E03240', 'E03270', 'E03300', 
         'E17500', 'E18400', 'E18500', 'E19200', 'E19800', 'E20100',
         'E20400', 'E32800']

predictors = cols[2:]

train = train[cols]
test = test[cols]

In [69]:
X_train = train[predictors]
X_test = test[predictors]

Y_train = train['P22250']
Y_test = test['P22250']

Y_train_sign = train['P22250_sign']
Y_test_sign = test['P22250_sign']

## Model

Train a random forests model.

In [70]:
# Reduce for faster runtime.
N_ESTIMATORS = 100
rf = ensemble.RandomForestRegressor(n_estimators=N_ESTIMATORS, 
                                    min_samples_leaf=1, random_state=3, 
                                    verbose=True, 
                                    n_jobs=-1)  # Use maximum number of cores.
rf.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.8min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=3, verbose=True, warm_start=False)

### Model description

Show the ten most important features.

In [71]:
feature_importance = pd.Series(rf.feature_importances_, index=X_train.columns)
feature_importance.sort_values(ascending=False)[:10]

E00300    0.176935
E00200    0.096906
E00600    0.087744
E19200    0.067132
MARS      0.062270
E18400    0.057161
E19800    0.056361
E20400    0.053500
E03300    0.050838
XTOT      0.050614
dtype: float64

## Predict

### Top-line (average)

In [72]:
pred = pd.DataFrame({'actual': Y_test,
                     'pred': rf.predict(X_test)})
pred['error'] = pred.pred - pred.actual
pred['actual_sign'] = np.sign(pred.actual)
pred['pred_sign'] = np.sign(pred.pred)
pred['correct_sign'] = (pred.actual_sign == pred.pred_sign)
pred['count'] = 1

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished


MAE, RMSE, and % negative/zero/positive.

In [73]:
pred.error.abs().mean()

35719.70797366644

In [74]:
pred.error.pow(2).mean() ** 0.5

303720.3695184425

In [75]:
pred.pivot_table(index='actual_sign', columns='pred_sign', values='count', 
                 aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1.0,2993,59,1623,4675
0.0,7626,11181,5177,23984
1.0,2438,43,1473,3954
All,13057,11283,8273,32613


In [76]:
pred.correct_sign.mean()

0.4797780026369852

### All trees

In [77]:
preds = []
for estimator in rf.estimators_:
    preds.append(estimator.predict(X_test))
preds = np.array(preds).transpose()  # One row per record.

In [78]:
rand_col = np.random.randint(N_ESTIMATORS, size=preds.shape[0])
random_tree = preds[np.arange(preds.shape[0]), rand_col]

In [79]:
pred_random_tree = pd.DataFrame({'actual': Y_test,
                                 'pred': random_tree})
pred_random_tree['error'] = pred_random_tree.pred - pred_random_tree.actual
pred_random_tree['actual_sign'] = np.sign(pred_random_tree.actual)
pred_random_tree['pred_sign'] = np.sign(pred_random_tree.pred)
pred_random_tree['correct_sign'] = (
    pred_random_tree.actual_sign == pred_random_tree.pred_sign)
pred_random_tree['count'] = 1

As expected, MAE and RMSE exceed values from the point estimate prediction.

In [80]:
pred_random_tree.error.abs().mean()

46945.67322697353

In [81]:
pred_random_tree.error.pow(2).mean() ** 0.5

471091.7919108312

But the distribution of sign is closer to correct, since it's not averaging out the zeros.

In [82]:
pred_random_tree.pivot_table(index='actual_sign', columns='pred_sign', 
                             values='count', aggfunc=sum, margins=True)

pred_sign,-1.0,0.0,1.0,All
actual_sign,,,,
-1.0,1843,1414,1418,4675
0.0,1625,20826,1533,23984
1.0,1450,1313,1191,3954
All,4918,23553,4142,32613


In [83]:
pred_random_tree.correct_sign.mean()

0.7316100941342409

#### Log-loss of sign

In [84]:
preds_neg = np.sum(preds < 0, axis=1) / 100
preds_zero = np.sum(preds == 0, axis=1) / 100
preds_pos = np.sum(preds > 0, axis=1) / 100

rf_pred_proba = list(map(list, zip(*[preds_neg, preds_zero, preds_pos])))

metrics.log_loss(Y_test_sign, rf_pred_proba)

0.6152340977951218

## Multinomial logistic regression of sign

In [85]:
mult = linear_model.LogisticRegression(
    multi_class='multinomial', solver='newton-cg', random_state=3)
mult.fit(X_train, Y_train_sign)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=3, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

### Coefficients

p-values are tricky to calculate, so just show the features with highest absolute coefficient on the zero class.

In [86]:
mult_coef = pd.DataFrame({
    'negative': mult.coef_[0],
    'zero': mult.coef_[1],
    'positive': mult.coef_[2],
    'abs_zero': np.abs(mult.coef_[1])},
    index=X_train.columns)

mult_coef.sort_values('abs_zero', ascending=False)[:10]

,abs_zero,negative,positive,zero
XTOT,0.064951,-0.031763,-0.033188,0.064951
MARS,0.064169,-0.031563,-0.032605,0.064169
E01100,0.017820,-0.009079,-0.008741,0.017820
EIC,0.009843,-0.004964,-0.004878,0.009843
DSI,0.002727,-0.001370,-0.001356,0.002727
E03210,0.001501,-0.000981,-0.000520,0.001501
E32800,0.000225,-0.000099,-0.000127,0.000225
E02300,0.000141,-0.000056,-0.000085,0.000141
E03150,0.000034,-0.000015,-0.000019,0.000034
E00800,0.000031,0.000012,-0.000042,0.000031


### Predict

In [87]:
mult_pred = pd.DataFrame({'actual': Y_test_sign,
                          'pred': mult.predict(X_test)})
mult_pred['sign_correct'] = (mult_pred.actual == mult_pred.pred)
mult_pred['count'] = 1
mult_pred.sign_correct.mean()

0.7689571643209763

In [88]:
mult_pred.pivot_table(index='actual', columns='pred', 
                      values='count', aggfunc=sum, margins=True)

pred,-1,0,1,All
actual,,,,
-1,1703,2858,114,4675
0,664,23256,64,23984
1,1273,2562,119,3954
All,3640,28676,297,32613


#### Log-loss

In [89]:
mult_pred_proba = mult.predict_proba(X_test)
metrics.log_loss(Y_test_sign, mult_pred_proba)

0.8627748111016034